In [1]:
# !pip install folktables

In [2]:
#info about Data
# https://arxiv.org/pdf/2108.04884.pdf

#ACSDataSource ,ACSEmployment, ACSPublicCoverage, ACSIncome

-> (using ACSEmployment) from the paper: predict whether an individual is employed, after filtering the ACS PUMS data
sample to only include individuals between the ages of 16 and 90  

In [1]:
# !pip install torchsummary

In [2]:
# !python --version

In [3]:
import pandas as pd
import numpy as np
import copy

import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

In [4]:
# detail about the state: load_acs.py

In [5]:

from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [6]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [7]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0


In [8]:
label

,ESR
0,False
1,False
2,False
3,False
4,False
...,...
47772,True
47773,False
47774,False
47775,False


In [9]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False


In [10]:
len(merged_df)

47777

In [11]:
group.value_counts()

RAC1P
1        35551
2        10115
9          835
6          556
8          491
3          182
5           33
7           13
4            1
dtype: int64

In [12]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

1.0    35551
2.0    10115
9.0      835
6.0      556
8.0      491
3.0      182
5.0       33
7.0       13
4.0        1
Name: RAC1P, dtype: int64

In [13]:
pd.unique(merged_df["RAC1P"])

array([1., 2., 8., 9., 6., 3., 5., 7., 4.])

In [14]:
#  for Black and white people
filtered_df= merged_df[["SEX","RAC1P","ESR"]][merged_df['RAC1P'].isin([1,2])]
print("len(filtered_df): ", len(filtered_df),"\n")
print(filtered_df["ESR"].value_counts(),"\n")
print(filtered_df["SEX"].value_counts(),"\n")
print(filtered_df["RAC1P"].value_counts(),"\n")
filtered_df.head() 

len(filtered_df):  45666 

False    26881
True     18785
Name: ESR, dtype: int64 

2.0    23737
1.0    21929
Name: SEX, dtype: int64 

1.0    35551
2.0    10115
Name: RAC1P, dtype: int64 



,SEX,RAC1P,ESR
0,2.0,1.0,False
1,2.0,2.0,False
2,1.0,1.0,False
3,1.0,1.0,False
4,2.0,1.0,False


In [15]:
# 1- Male 2- Female
# 1- White 2-Black

counts_frq = filtered_df.groupby(['SEX', 'RAC1P']).size()
counts_frq

SEX  RAC1P
1.0  1.0      17309
     2.0       4620
2.0  1.0      18242
     2.0       5495
dtype: int64

In [16]:
merged_df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,False


In [17]:
# # filter the dataframe for each combination

#White Men
WM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 1)]
#Black Men
BM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 2)]
#white Women
WW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 1)]
#Black Women
BW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 2)]

In [18]:
WM

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
7,38.0,12.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,False
8,41.0,16.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
14,21.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47768,76.0,13.0,3.0,0.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,False
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,False


# Removing Sex and Gen from 4 Groups

In [19]:
WM.columns

Index(['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
      dtype='object')

In [20]:
WM_updated=WM[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM', 'ESR']]
BM_updated=BM[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM', 'ESR']]
WW_updated=WW[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM', 'ESR']]
BW_updated=BW[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM', 'ESR']]

# Grouping

In [21]:
#Group-1
#Train set: White men+black men+Black women—- validation set: black men
WM_BM_BW_train = pd.concat([WM_updated,BM_updated,BW_updated])
BM_validation=BM_updated.copy()

#Group-2
#Train set: White women+black women+black men—- validation set: black women
WW_BW_BM_train=pd.concat([WW_updated,BW_updated,BM_updated])
BW_validation=BW_updated.copy()

#Group-3
#Train set: black men+white men+white women+validation set: white men
BM_WM_WW_train=pd.concat([BM_updated,WM_updated,WW_updated])
WM_validation=WM_updated.copy()

#Group-4
#Train set: white women+white men+black women, validation set: white women
WW_WM_BW_train=pd.concat([WW_updated,WM_updated,BW_updated])
WW_validation=WW_updated.copy()

In [22]:

print("BM" ,len(BM),"RAC: ", pd.unique(BM['RAC1P']),"SEX: ", pd.unique(BM['SEX']))
print("BW" ,len(BW),"RAC: ", pd.unique(BW['RAC1P']),"SEX: ", pd.unique(BW['SEX']))
print("WM" ,len(WM),"RAC: ", pd.unique(WM['RAC1P']),"SEX: ", pd.unique(WM['SEX']))
print("WW" ,len(WW),"RAC: ", pd.unique(WW['RAC1P']),"SEX: ", pd.unique(WW['SEX']))

BM 4620 RAC:  [2.] SEX:  [1.]
BW 5495 RAC:  [2.] SEX:  [2.]
WM 17309 RAC:  [1.] SEX:  [1.]
WW 18242 RAC:  [1.] SEX:  [2.]


In [23]:
data_to_use_train1= WM_BM_BW_train
data_to_use_val1= BM_validation

data_to_use_train2= WW_BW_BM_train
data_to_use_val2= BW_validation


data_to_use_train3= BM_WM_WW_train
data_to_use_val3= WM_validation


data_to_use_train4= WW_WM_BW_train
data_to_use_val4= WW_validation

In [24]:

X_train1= data_to_use_train1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train1= data_to_use_train1[['ESR']]


X_train2= data_to_use_train2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train2= data_to_use_train2[['ESR']]

X_train3= data_to_use_train3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train3= data_to_use_train3[['ESR']]

X_train4= data_to_use_train4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train4= data_to_use_train4[['ESR']]

# X_train = torch.from_numpy(X_train.to_numpy()).float()
# y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())


In [30]:
X_valid1 = data_to_use_val1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid1 = data_to_use_val1[['ESR']]

X_valid2 = data_to_use_val2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid2 = data_to_use_val2[['ESR']]

X_valid3 = data_to_use_val3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid3 = data_to_use_val3[['ESR']]

X_valid4 = data_to_use_val4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid4 = data_to_use_val4[['ESR']]

# X_valid = torch.from_numpy(X_valid.to_numpy()).float()
# y_valid = torch.squeeze(torch.from_numpy(y_valid.to_numpy()).float())

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [31]:
print("TYPE X: ",type(X_train1)," Type Y", type(y_train1),"\n")
print("len(X_train),len(y_train), len(X_val),len(y_val): ",len(X_train1),len(y_train1), len(X_valid1),len(y_valid1))
print("X_train.shape, y_train.shape ",X_train1.shape, y_train1.shape,"\n")
print("X_valid.shape, y_valid.shape ",X_valid1.shape, y_valid1.shape,"\n")

X_train1

TYPE X:  <class 'numpy.ndarray'>  Type Y <class 'pandas.core.frame.DataFrame'> 

len(X_train),len(y_train), len(X_val),len(y_val):  27424 27424 4620 4620
X_train.shape, y_train.shape  (27424, 14) (27424, 1) 

X_valid.shape, y_valid.shape  (4620, 14) (4620, 1) 



array([[0.90734272, 0.29103446, 0.08559837, ..., 0.03423935, 0.03423935,
        0.01711967],
       [0.73531602, 0.49896444, 0.13130643, ..., 0.05252257, 0.05252257,
        0.05252257],
       [0.86858561, 0.27429019, 0.11428758, ..., 0.02285752, 0.04571503,
        0.04571503],
       ...,
       [0.74337629, 0.62443609, 0.14867526, ..., 0.0594701 , 0.0594701 ,
        0.0594701 ],
       [0.98263061, 0.16983739, 0.01213124, ..., 0.02426248, 0.02426248,
        0.02426248],
       [0.95612824, 0.26838687, 0.05032254, ..., 0.03354836, 0.03354836,
        0.03354836]])

In [32]:
y_train1

,ESR
2,False
3,False
7,False
8,True
14,False
...,...
47710,True
47750,False
47751,True
47764,False


In [33]:
X_valid1 = preprocessing.normalize(X_valid1)
X_train1 = preprocessing.normalize(X_train1)

X_valid2 = preprocessing.normalize(X_valid2)
X_train2 = preprocessing.normalize(X_train2)

X_valid3 = preprocessing.normalize(X_valid3)
X_train3 = preprocessing.normalize(X_train3)

X_valid4 = preprocessing.normalize(X_valid4)
X_train4 = preprocessing.normalize(X_train4)


In [34]:
encoder = LabelEncoder()


In [35]:
# Binary encoding of labels
encoder.fit(y_train1)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [36]:
y_train1 = encoder.transform(y_train1)
y_valid1 = encoder.transform(y_valid1)

y_train2 = encoder.transform(y_train2)
y_valid2 = encoder.transform(y_valid2)

y_train3 = encoder.transform(y_train3)
y_valid3 = encoder.transform(y_valid3)

y_train4 = encoder.transform(y_train4)
y_valid4 = encoder.transform(y_valid4)


C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Lo

In [37]:
# Convert to 2D PyTorch tensors
X_train1= torch.tensor(X_train1, dtype=torch.float32)
X_valid1= torch.tensor(X_valid1, dtype=torch.float32)
y_train1 = torch.tensor(y_train1, dtype=torch.float32).reshape(-1, 1)
y_valid1 = torch.tensor(y_valid1, dtype=torch.float32).reshape(-1, 1)


X_train2 = torch.tensor(X_train2, dtype=torch.float32)
X_valid2 = torch.tensor(X_valid2, dtype=torch.float32)
y_train2 = torch.tensor(y_train2, dtype=torch.float32).reshape(-1, 1)
y_valid2 = torch.tensor(y_valid2, dtype=torch.float32).reshape(-1, 1)


X_train3= torch.tensor(X_train3, dtype=torch.float32)
X_valid3= torch.tensor(X_valid3, dtype=torch.float32)
y_train3 = torch.tensor(y_train3, dtype=torch.float32).reshape(-1, 1)
y_valid3 = torch.tensor(y_valid3, dtype=torch.float32).reshape(-1, 1)



X_train4= torch.tensor(X_train4, dtype=torch.float32)
X_valid4= torch.tensor(X_valid4, dtype=torch.float32)
y_train4 = torch.tensor(y_train4, dtype=torch.float32).reshape(-1, 1)
y_valid4 = torch.tensor(y_valid4, dtype=torch.float32).reshape(-1, 1)

In [39]:
X_train1,y_train1

(tensor([[0.9073, 0.2910, 0.0856,  ..., 0.0342, 0.0342, 0.0171],
         [0.7353, 0.4990, 0.1313,  ..., 0.0525, 0.0525, 0.0525],
         [0.8686, 0.2743, 0.1143,  ..., 0.0229, 0.0457, 0.0457],
         ...,
         [0.7434, 0.6244, 0.1487,  ..., 0.0595, 0.0595, 0.0595],
         [0.9826, 0.1698, 0.0121,  ..., 0.0243, 0.0243, 0.0243],
         [0.9561, 0.2684, 0.0503,  ..., 0.0335, 0.0335, 0.0335]]),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [1.],
         [0.],
         [0.]]))

In [40]:
y_train1,y_valid1

(tensor([[0.],
         [0.],
         [0.],
         ...,
         [1.],
         [0.],
         [0.]]),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]))

In [41]:
X_train1.shape, y_train1.shape

(torch.Size([27424, 14]), torch.Size([27424, 1]))

# data loader-torch

In [42]:
from torch.utils.data import DataLoader, Subset, random_split
import torch.utils.data as data_utils
import pickle
# Passing to DataLoader

train1 = data_utils.TensorDataset(X_train1, y_train1)
valid1 = data_utils.TensorDataset(X_valid1, y_valid1)

train2 = data_utils.TensorDataset(X_train2, y_train2)
valid2 = data_utils.TensorDataset(X_valid2, y_valid2)

train3 = data_utils.TensorDataset(X_train3, y_train3)
valid3 = data_utils.TensorDataset(X_valid3, y_valid3)

train4 = data_utils.TensorDataset(X_train4, y_train4)
valid4 = data_utils.TensorDataset(X_valid4, y_valid4)

trainloader1 = data_utils.DataLoader(train1, batch_size=10, shuffle=True)
trainloader2 = data_utils.DataLoader(train2, batch_size=10, shuffle=True)
trainloader3 = data_utils.DataLoader(train3, batch_size=10, shuffle=True)
trainloader4 = data_utils.DataLoader(train4, batch_size=10, shuffle=True)

valloader1 = data_utils.DataLoader(valid1, batch_size=10)
valloader2 = data_utils.DataLoader(valid2, batch_size=10)
valloader3 = data_utils.DataLoader(valid3, batch_size=10)
valloader4 = data_utils.DataLoader(valid4, batch_size=10)




In [43]:
train_data_all_client=[trainloader1,trainloader2,trainloader3,trainloader4]
val_data_all_client=[valloader1,valloader2,valloader3,valloader4]

In [44]:
with open("train_1_client.pkl", "wb") as f:
    pickle.dump(train_data_all_client, f)
    
with open("valid_1_client.pkl", "wb") as f:
    pickle.dump(val_data_all_client, f)

In [76]:
all_data=[trainloader,trainloader,trainloader,trainloader]

with open("chirag_data.pkl", "wb") as f:
    pickle.dump(all_data, f)

In [68]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

In [69]:
def model_train(model, X_train, y_train, X_val, y_val):
    # Loss function and optimizer
    loss_fn = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Number of epochs and batch size
    n_epochs = 10
    batch_size = 1000

    # Initialize best accuracy and weights
    best_acc = -np.inf
    best_weights = None

    # Train the model
    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(range(len(X_train) // batch_size), unit="batch", mininterval=0, disable=True) as bar:
            for i in bar:
                # Get a batch of data
                X_batch = X_train[i * batch_size:(i + 1) * batch_size]
                y_batch = y_train[i * batch_size:(i + 1) * batch_size]

                # Forward pass
                y_pred = model(X_batch)

                # Calculate the loss
                loss = loss_fn(y_pred, y_batch)

                # Backpropagate the loss
                optimizer.zero_grad()
                loss.backward()

                # Update the weights
                optimizer.step()

                # Calculate the accuracy
                acc = (y_pred.round() == y_batch).float().mean()

                # Update the best accuracy and weights
                if acc > best_acc:
                    best_acc = acc
                    best_weights = copy.deepcopy(model.state_dict())

                # Track training loss
                train_loss = loss.item()

                # Evaluate the model on the validation set
                model.eval()
                y_pred = model(X_val)
                val_acc = (y_pred.round() == y_val).float().mean()

                # Track validation loss
                val_loss = loss_fn(y_pred, y_val).item()

        print("Epoch: %d, Training Loss: %.4f, Validation Loss: %.4f" % (epoch, train_loss, val_loss))

    # Return the best accuracy and validation accuracy
    return best_acc, val_acc



kfold = StratifiedKFold(n_splits=2, shuffle=True)

cv_scores_deep = []
cv_val_scores_deep = []

for train, test in kfold.split(X_train, y_train):
    
    model = DeepNet()
    #model = Deep_wide_Net()
    acc, val_acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (deep): %.2f, Validation Acc: %.2f" % (acc, val_acc))
    cv_scores_deep.append(acc)
    cv_val_scores_deep.append(val_acc)
    
    
deep_acc = np.mean(cv_scores_deep)
deep_std = np.std(cv_scores_deep)
deep_val_acc = np.mean(cv_val_scores_deep)
deep_val_std = np.std(cv_val_scores_deep)
print("Deep: %.2f%% (+/- %.2f%%)" % (deep_acc*100, deep_std*100))
print("Deep Validation: %.2f%% (+/- %.2f%%)" % (deep_val_acc*100, deep_val_std*100))

Epoch: 0, Training Loss: 0.6468, Validation Loss: 0.6594
Epoch: 1, Training Loss: 0.6141, Validation Loss: 0.5916
Epoch: 2, Training Loss: 0.5608, Validation Loss: 0.5433
Epoch: 3, Training Loss: 0.5110, Validation Loss: 0.5282
Epoch: 4, Training Loss: 0.5026, Validation Loss: 0.5274
Epoch: 5, Training Loss: 0.5075, Validation Loss: 0.5250
Epoch: 6, Training Loss: 0.4951, Validation Loss: 0.5303
Epoch: 7, Training Loss: 0.4765, Validation Loss: 0.4805
Epoch: 8, Training Loss: 0.5124, Validation Loss: 0.4598
Epoch: 9, Training Loss: 0.5071, Validation Loss: 0.4601
Accuracy (deep): 0.81, Validation Acc: 0.78
Epoch: 0, Training Loss: 0.6165, Validation Loss: 0.6455
Epoch: 1, Training Loss: 0.5925, Validation Loss: 0.5768
Epoch: 2, Training Loss: 0.5416, Validation Loss: 0.5422
Epoch: 3, Training Loss: 0.4891, Validation Loss: 0.5376
Epoch: 4, Training Loss: 0.4842, Validation Loss: 0.5468
Epoch: 5, Training Loss: 0.4743, Validation Loss: 0.5563
Epoch: 6, Training Loss: 0.4683, Validation 

In [73]:
model = DeepNet()

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate

# Move the model to the appropriate device (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, targets)
        
        # Backpropagation
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}")

# Validation loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in valloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        
        # You can define your own accuracy calculation here
        # For binary classification, you can use something like this:
        predicted = outputs > 0.5
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.2%}")

Epoch [1/10], Batch [10/2743], Loss: 0.6867
Epoch [1/10], Batch [20/2743], Loss: 0.7185
Epoch [1/10], Batch [30/2743], Loss: 0.6457
Epoch [1/10], Batch [40/2743], Loss: 0.5483
Epoch [1/10], Batch [50/2743], Loss: 0.5523
Epoch [1/10], Batch [60/2743], Loss: 0.9352
Epoch [1/10], Batch [70/2743], Loss: 0.5671
Epoch [1/10], Batch [80/2743], Loss: 0.4657
Epoch [1/10], Batch [90/2743], Loss: 0.6209
Epoch [1/10], Batch [100/2743], Loss: 0.5861
Epoch [1/10], Batch [110/2743], Loss: 0.4733
Epoch [1/10], Batch [120/2743], Loss: 0.6986
Epoch [1/10], Batch [130/2743], Loss: 0.5486
Epoch [1/10], Batch [140/2743], Loss: 0.6310
Epoch [1/10], Batch [150/2743], Loss: 0.7371
Epoch [1/10], Batch [160/2743], Loss: 0.5165
Epoch [1/10], Batch [170/2743], Loss: 0.5230
Epoch [1/10], Batch [180/2743], Loss: 0.6340
Epoch [1/10], Batch [190/2743], Loss: 0.5800
Epoch [1/10], Batch [200/2743], Loss: 0.6721
Epoch [1/10], Batch [210/2743], Loss: 0.5257
Epoch [1/10], Batch [220/2743], Loss: 0.5481
Epoch [1/10], Batch

KeyboardInterrupt: 

In [34]:
# name_of_model='WM_BM_BW_deep_wide.pth'

# WM_BM_BW_train done
# WW_BW_BM_train done
# BM_WM_WW_train done
# WW_WM_BW_train done

# name_of_model='WW_WM_BW.pth'

In [35]:
# torch.save(model.state_dict(), name_of_model)

In [113]:
merged_df_test.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [112]:
counts_frq_test = merged_df_test.groupby(['SEX', 'RAC1P']).size()
counts_frq_test

SEX  RAC1P
1.0  1.0      16783
     2.0       4988
     3.0        110
     4.0          1
     5.0         28
     6.0        290
     7.0          8
     8.0        253
     9.0        395
2.0  1.0      18044
     2.0       5653
     3.0         97
     4.0          1
     5.0         19
     6.0        327
     7.0          7
     8.0        215
     9.0        426
dtype: int64

In [111]:
WM_BM_BW_test= pd.concat([WM_test,BM_test,BW_test])

WW_BW_BM_test= pd.concat([WW_test,BW_test,BM_test])

BM_WM_WW_test= pd.concat([WM_test,WM_test,WW_test])

WW_WM_BW_test= pd.concat([WW_test,WM_test,BW_test])